Using kernel `conda_pytorch_latest_p36`

# Import

In [ ]:
import sys
sys.path.append('../../../')

In [ ]:
from pathlib import Path
import os
import random
import json
from datetime import datetime

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import torch
import sagemaker
from sagemaker import get_execution_role
import boto3
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, 
    TrainingArguments, Trainer
)

In [ ]:
from deep.constants import *
from deep.utils import *

In [ ]:
%load_ext autoreload
%autoreload 2

## Data

In [ ]:
train = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_train.csv')
val = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_val.csv')
test = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_test.csv')

In [ ]:
columns_to_keep = ['excerpt'] + DIMENSION_CLASSES

In [ ]:
def process_for_sector(df, sector):
    df = df[['excerpt', sector]]
        
    return df

In [ ]:
train_df = process_for_sector(train, 'Humanitarian Conditions')
test_df = process_for_sector(test, 'Humanitarian Conditions')

In [ ]:
sample = True

if sample:
    train_df = train_df.sample(100)
    test_df = test_df.sample(100)

In [ ]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
train_encodings = tokenizer(list(train_df.excerpt), truncation=True, padding=True)
test_encodings = tokenizer(list(test_df.excerpt), truncation=True, padding=True)

In [ ]:
train_labels = list(train_df["Humanitarian Conditions"])
test_labels = list(test_df["Humanitarian Conditions"])

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
test_dataset = Dataset(test_encodings, test_labels)


In [ ]:
# define training args
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    evaluation_strategy="epoch",
    logging_steps=1,
)

# create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# train model
trainer.train()


## Sagemaker Prep

### Session

In [ ]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = 'AmazonSageMaker-ExecutionRole-20210519T102514'

### Bucket upload

In [ ]:
sample = False

if sample:
    train_df = train_df.sample(1000)
    test_df = test_df.sample(1000)

    
job_name = f"pytorch-training-{formatted_time()}"
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name

s3_train_data = str(input_path / 'train_df.pickle')
s3_validation_data = str(input_path / 'test_df.pickle')

train_df.to_pickle(s3_train_data, protocol=4)
test_df.to_pickle(s3_validation_data, protocol=4)

### Estimator Definition

In [ ]:
instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

In [ ]:
metric_definitions=[
    {'Name': 'loss', 'Regex': "'loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'learning_rate', 'Regex': "'learning_rate': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_loss', 'Regex': "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_accuracy', 'Regex': "'eval_accuracy': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_f1', 'Regex': "'eval_f1': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_precision', 'Regex': "'eval_precision': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_recall', 'Regex': "'eval_recall': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_stupid_metric', 'Regex': "'eval_stupid_metric': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_runtime', 'Regex': "'eval_runtime': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_samples_per_second', 'Regex': "'eval_samples_per_second': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'epoch', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"}]

In [ ]:
# # set True if you need spot instance
# use_spot = True
# train_max_run_secs =   2*24 * 60 * 60
# spot_wait_sec =  5 * 60
# max_wait_time_secs = train_max_run_secs +  spot_wait_sec

# if not use_spot:
#     max_wait_time_secs = None
    
# # During local mode, no spot.., use smaller dataset
# if instance_type == 'local':
#     use_spot = False
#     max_wait_time_secs = 0
#     wait = True
#     # Use smaller dataset to run locally
#     inputs = inputs_sample


In [ ]:
from sagemaker.pytorch import PyTorch

hyperparameters={'epochs': 2,
                 'train_batch_size': 32,
                 'model_name': 'distilbert-base-uncased'
                 }

estimator = PyTorch(
    entry_point='train.py',
    source_dir=str(SCRIPTS_TRAINING_PATH / 'stefano/pytorch_estimator_example'),
    output_path=str(DEV_BUCKET / 'models/'),
    code_location=str(input_path),
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    framework_version='1.8',
    py_version='py36',
    hyperparameters = hyperparameters,
    metric_definitions=metric_definitions,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [ ]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [ ]:
estimator.fit(fit_arguments, job_name=job_name, wait=False)

In [ ]:
from sagemaker import TrainingJobAnalytics

# Captured metrics can be accessed as a Pandas dataframe
df = TrainingJobAnalytics(training_job_name=estimator.latest_training_job.name).dataframe()
df.head(10)

In [ ]:
df